In [3]:
import pandas as pd
import numpy as np
import datetime

In [15]:

path = "funds_needed.xlsx"
def load_data(path):
    try:
        data = pd.read_excel(path, sheet_name='Sheet1')
        
    except:
        data = pd.read_csv(path, usecols= ['P/E', 'P/B', 'Date'])
        
    return data #put the path here
data = load_data(path)

In [131]:
import os
# Define the path to the folder
folder_path = "unprocessed"
# Loop through each file in the folder
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and "unprocessed" in filename:
            print(file_path)
            # Check if it's a file (not a folder)
            if os.path.isfile(file_path):
                data =  load_data(file_path)
                print("data",data)
                data = preprocess_dates(data)
                data = preprocess_missing(data)
                data.to_excel(file_path+'.xlsx', index = False)


else:
    print(f"The folder '{folder_path}' does not exist.")


unprocessed\index_unprocessed.xlsx
data         dates  ICICI Pru Nifty 50 Index Fund-Reg(G)
0  2024-10-28                              243.0207
1  2024-10-29                              244.4600
2  2024-10-30                              243.1995
3  2024-10-31                              241.8910
4  2024-11-04                              239.7837
5  2024-11-05                              241.9548
6  2024-11-06                              244.7113
7  2024-11-07                              241.8790
8  2024-11-08                              241.3654
9  2024-11-11                              241.2887
10 2024-11-12                              238.7097
11 2024-11-13                              235.4656
12 2024-11-14                              235.2435
13 2024-11-18                              234.4459
14 2024-11-19                              235.0884
15 2024-11-21                              233.4025
16 2024-11-22                              238.9674
17 2024-11-25           

In [97]:
path = "NIFTY50 VALUE 20_Historical_PE_PB_DIV_Data_01102024to12012025.csv"
data = pd.read_csv(path)

In [16]:
data.isna().mean()

dates                        0.000000
Kotak Gold Fund(G)           0.016071
SBI Contra Fund-Reg(IDCW)    0.000402
dtype: float64

In [17]:
print(data.iloc[0])
print(data.iloc[-1])

dates                        2015-03-12 00:00:00
Kotak Gold Fund(G)                       11.5101
SBI Contra Fund-Reg(IDCW)              234.22778
Name: 0, dtype: object
dates                        2025-04-11 00:00:00
Kotak Gold Fund(G)                       36.1298
SBI Contra Fund-Reg(IDCW)             893.207019
Name: 2488, dtype: object


In [18]:
#start, end date format id mm/dd/yy
def preprocess_dates(data, start='01/01/2025',  end='3/31/2025'):
    if "dates" not in list(data.columns):
        date_col = list(set(data.columns[data.columns.str.contains('ate')].values))
        data.rename(columns = {date_col[0]:'dates'}, inplace=True)
    
    dates = pd.DataFrame()
    dates['dates'] = list(set(pd.date_range(start=start, end=end, freq='B'))) #for skipping weekends use "B"
    dates.sort_values(by = 'dates', inplace = True) #business/trading dates in this range
    dates.dates = pd.to_datetime(dates.dates).dt.date
    data = data.loc[:, ~data.T.duplicated()]
    
    data.drop_duplicates(subset=['dates'], keep='first', inplace=True)
    data.dates = pd.to_datetime(data.dates).dt.date
    data.sort_values(by = 'dates', inplace=True)
    print(data)
    data = pd.merge(dates, data, on = 'dates', how = 'left')
    print(data)
    data.sort_values(by = 'dates', inplace=True)
    
   

    return data


In [19]:
def preprocess_missing(data):
     for col in data.columns:
          if col !=  'dates':
               data[col] = data[col].ffill().add(data[col].bfill()).div(2)

     return data


In [20]:
data = preprocess_dates(data)

data = preprocess_missing(data)

#export to excel
data.to_excel(path[:-5]+'.xlsx', index = False)



           dates  Kotak Gold Fund(G)  SBI Contra Fund-Reg(IDCW)
0     2015-03-12             11.5101                 234.227780
1     2015-03-13             11.5157                 231.315972
2     2015-03-16             11.4984                 231.248549
3     2015-03-17             11.4152                 232.730790
4     2015-03-18             11.3748                 232.981517
...          ...                 ...                        ...
2484  2025-04-04             34.7090                 894.644327
2485  2025-04-07             34.2912                 870.041097
2486  2025-04-08             34.1344                 882.027474
2487  2025-04-09             34.7220                 880.549017
2488  2025-04-11             36.1298                 893.207019

[2489 rows x 3 columns]
         dates  Kotak Gold Fund(G)  SBI Contra Fund-Reg(IDCW)
0   2025-01-01             29.9926                 961.010293
1   2025-01-02             30.0701                 970.971507
2   2025-01-03       

In [21]:
data.isna().mean()

dates                        0.0
Kotak Gold Fund(G)           0.0
SBI Contra Fund-Reg(IDCW)    0.0
dtype: float64

In [22]:
# data['P/E'] = pd.to_numeric(data['P/E'], errors = 'coerce')

In [23]:
data.dtypes

dates                         object
Kotak Gold Fund(G)           float64
SBI Contra Fund-Reg(IDCW)    float64
dtype: object

In [24]:

data = preprocess_missing(data)

#export to excel
data.to_excel(path[:-5]+'.xlsx', index = False)

In [25]:
data.isna().mean()

dates                        0.0
Kotak Gold Fund(G)           0.0
SBI Contra Fund-Reg(IDCW)    0.0
dtype: float64

In [26]:
data

,dates,Kotak Gold Fund(G),SBI Contra Fund-Reg(IDCW)
0,2025-01-01,29.9926,961.010293
1,2025-01-02,30.0701,970.971507
2,2025-01-03,30.2535,968.136574
3,2025-01-06,30.0547,950.822754
4,2025-01-07,30.1049,954.307269
...,...,...,...
59,2025-03-25,34.0157,918.584749
60,2025-03-26,33.9693,912.757630
61,2025-03-27,34.2076,916.974024
62,2025-03-28,34.5466,913.430724
